# Asignación y Desenmascarado

Saquemos las máscaras a los tweets

In [1]:
import pandas as pd

df_titulares = pd.read_csv("../data/titulos_filtrados.csv")

df_titulares = df_titulares.rename(columns={"Unnamed: 0": "id"}).set_index("id")

df_titulares

,medio,fecha,titulo
id,,,
23,la izquierda diario,2019-09-21 00:00:00,Barrio Aeroclub: la Izquierda propone urbaniza...
24,la izquierda diario,2019-09-21 00:00:00,Elecciones municipales: el #FITUnidad busca fo...
31,la izquierda diario,2019-09-21 00:00:00,[Videos] Mirá todos los spots del Frente de Iz...
65,la nación,2019-09-22 00:00:00,Tipo de cambio: por qué nos preocupa el dólar ...
75,la izquierda diario,2019-09-22 00:00:00,Brutalidad de la Policía de Urtubey en el reci...
...,...,...,...
22608,prensa obrera,2019-10-27 23:05:46,Declaración del Partido Obrero-Frente de Izqui...
22611,la nación,2019-10-27 23:16:00,"Resultados elecciones 2019: Alberto Fernández,..."
22612,la nación,2019-10-27 23:21:00,Elecciones 2019. La llamada de Macri a Alberto...


In [2]:
import pandas as pd
import glob
import numpy as np
import re



paths = glob.glob("../data/etiquetados_final/*.xlsx")

dfs = []
annotators = []

for path in paths:
    name = os.path.basename(path).split(".")[0]
    
    df = pd.read_excel(path)
    df = df.iloc[3:]
    annotators.append(name)

    df.rename(columns={df.columns[0]: "id", "TITULO": "titulo_mask", " ": "etiquetado", "Etiquetado?": "etiquetado"}, inplace=True)

    # Por algún motivo perdimos este titular...
    df.set_index("id", inplace=True)

    ids = [x for x in df.index if x not in {39}]
    
    df.loc[ids, "titulo"] = df_titulares.loc[ids, "titulo"]
    # KJJJJ qué desastre esto
    df.loc[39, "titulo"] = "Entre el milagro de Macri y la grieta del peronismo"
    df["anotador"] = name
    df.reset_index(inplace=True)
    dfs.append(df)
    
annotators = sorted(annotators)
annotator_df = pd.concat(dfs).set_index(["id", "anotador"])

## Asignación

Primero, veamos cómo asignarlo

In [3]:
ids = dfs[0]["id"]

label_cols = ['TARGET A', 'TARGET B', 'TARGET C', 'TARGET D',
       'TARGET E', 'TARGET F']

total = 0

accepted_ids = []

for _, row in dfs[0].iterrows():
    idx = row["id"]
    labels = annotator_df.loc[idx]

    assert len(labels) == 3
    annotated = labels[label_cols].notna()

    """
    Chequeamos si la anotación difirió
    """
    if not ( all(annotated.iloc[0] == annotated.iloc[1]) and all(annotated.iloc[1] == annotated.iloc[2])):
        """
        print("="*80 + "\n" * 2)
        print(idx, row["titulo"])
        print(annotated.sum(axis=1))
        print(annotated)
        """
        total+=1
    else:
        accepted_ids.append(idx)

Ok, quedémonos con accepted_ids

Para cada titular, hagamos lo siguiente:

- Votación mayoritaria (elegimos al que tenga 2 votos)
- Si no ocurre...mmm, tiremos el titular mejor

In [4]:
import re
from tqdm.auto import tqdm

def get_targets(instance):
    mascarado = re.sub(r"\s+", " ", instance["titulo_mask"]).strip()
    titulo = instance["titulo"]

    i = 0
    j = 0

    found_targets = {}

    while len(found_targets) < len(instance["targets"]) and i < len(titulo):
        if titulo[i] == mascarado[j]:
            i += 1
            j += 1
        else:
            "son distintos, tiene que ser un target"
            assert mascarado[j] == "["
            
            next_idx = mascarado[j:].index("]") + j 
            
            target = mascarado[j+1:next_idx]

            j = next_idx + 1

            target_name = ""
            if j >= len(mascarado)-1:
                """
                Listo, me quedo con todo
                """
                target_name = titulo[i:]
                i = len(titulo)
            else:
                """
                Todavía queda algo
                """
                
                pattern = mascarado[j:j+2]
                next = titulo[i:].index(pattern)
                target_name = titulo[i:i+next]
                i = i + next 
            
            if target in found_targets:
                found_targets[target] = [found_targets[target], target_name]
            found_targets[target] = target_name
    return found_targets

problematic = []

dataset = []

for idx in tqdm(accepted_ids):
    annotated_columns = [col for col, annotated in annotator_df.loc[idx, label_cols].notna().all().items() if annotated]

    # Agarro un etiquetador, total da igual
    row = annotator_df.loc[(idx, "et1")]

    ret = {k:row[k] for k in ["titulo", "titulo_mask"]}
    ret["id"] = idx
    mascara = re.sub(r"\s+", " ", ret["titulo_mask"]).strip()
    mascara = re.sub(r"(\])\s+([^\w\s“])", r"\1\2", mascara)

    ret["titulo_mask"] = mascara 
    
    ret["targets"] = {}
    for col in annotated_columns:
        counts = annotator_df.loc[idx, col].value_counts()

        if counts.max() == 1:
            problematic.append(idx)
        else:
            label = counts.index[counts.argmax()]
            ret["targets"][col] = label


    if ret["targets"]:
        try:
            ret["mapping"] = get_targets(ret)
        except:
            pass
        dataset.append(ret)
print(len(problematic))

100%|██████████| 2183/2183 [00:09<00:00, 234.20it/s]

114


In [5]:
set_ids = []

def set_mapping(idx, mapping, keep=False):
    if keep:
        set_ids.append(idx)
    instance = next(x for x in dataset if x["id"] == idx)

    instance["mapping"] = mapping

set_mapping(639, {"TARGET E": "Alberto"})
set_mapping(1291, {"TARGET C": "Gobierno", "TARGET E": ["Alberto F.", "Cristina Kirchner"]}, keep=False)
set_mapping(2060, {"TARGET C": ["Gobierno", "Mauricio Macri"]}, keep=False)
set_mapping(2057, {"TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(2158, {"TARGET C": ["Gobierno", "Macri"]}, keep=False)
set_mapping(2423, {"TARGET C": "Gobierno", "TARGET E": "Alberto Fernández"}, keep=False)
set_mapping(4600, {"TARGET E": "Alberto"})
set_mapping(4783, {"TARGET A": "Espert", "TARGET D": "Nico del Caño"})
set_mapping(4885, {"TARGET B": "Roberto Lavagna", "TARGET C": "Gobierno"})
set_mapping(4936, {"TARGET B": ["Roberto Lavagna", "Consenso Federal"], "TARGET C": "Gobierno"})
set_mapping(5061, {"TARGET C": "Gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(5338, {"TARGET C": "Mauricio Macri", "TARGET E": "peronismo"})
set_mapping(5453, {"TARGET E": "Alberto"})
set_mapping(5070, {"TARGET A": "José Luis Espert", "TARGET D": "Nicolás del Caño"})
set_mapping(5804, {"TARGET E": "Alberto Fernández", "TARGET C": "gobierno"})
set_mapping(6014, {"TARGET B": "Roberto Lavagna", "TARGET C": "Gobierno"})
set_mapping(6605, {"TARGET C": "Gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(7429, {"TARGET C": "Gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(7725, {"TARGET C": "gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(8131, {"TARGET C": "gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(10204, {"TARGET E": "PJ"})
set_mapping(10513, {"TARGET B": "Roberto Lavagna", "TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(10748, {"TARGET C": "gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(11474, {"TARGET C": "gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(12462, {"TARGET C": "Gobierno", "TARGET E": "Cristina"})
set_mapping(14150, {"TARGET C": "Macri", "TARGET E": "Alberto F."})
set_mapping(15856, {"TARGET B": "Roberto Lavagna", "TARGET C": "Gobierno"})
set_mapping(16255, {"TARGET C": ["Gobierno", "Macri"]})
set_mapping(16134, {"TARGET B": "Lavagna", "TARGET C": "Gobierno"})
set_mapping(16424, {"TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(17935, {"TARGET C": "Gobierno", "TARGET E": "CFK"})
set_mapping(18449, {"TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(18467, {"TARGET A": "José Luis Espert", "TARGET C": "Mauricio Macri", "TARGET E": "kirchnerismo"})
set_mapping(19447, {"TARGET C": "Gobierno"})
set_mapping(20433, {"TARGET C": "Mauricio Macri"})
set_mapping(21233, {"TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(21483, {"TARGET C": "Gobierno", "TARGET E": "peronismo"})
set_mapping(22381, {"TARGET E": "Alberto Fernández"})
delete_ids = [2219]

dataset = [x for x in dataset if x["id"] not in delete_ids]

[x for x in dataset if "mapping" not in x or x["id"] in set_ids]

[]

In [6]:
annotator_df.loc[14150]

,titulo_mask,etiquetado,TARGET A,TARGET B,TARGET C,TARGET D,TARGET E,TARGET F,titulo
anotador,,,,,,,,,
e2,"[TARGET E] . le respondió a [TARGET C] por ""el...",SI,NaN,NaN,NEG,NaN,POS,NaN,"Alberto F. le respondió a Macri por ""el dedito..."
e3,"[TARGET E] . le respondió a [TARGET C] por ""el...",SI,NaN,NaN,POS,NaN,NEG,NaN,"Alberto F. le respondió a Macri por ""el dedito..."
et1,"[TARGET E] . le respondió a [TARGET C] por ""el...",SI,NaN,NaN,NEG,NaN,NEU,NaN,"Alberto F. le respondió a Macri por ""el dedito..."


In [7]:
instance

NameError: name 'instance' is not defined

In [ ]:
clean_dataset[0]

{'titulo': 'Barrio Aeroclub: la Izquierda propone urbanización y adjudicación de terrenos',
 'id': 23,
 'labels': {'l': 'POS',
  'a': 'POS',
  ' ': 'POS',
  'I': 'POS',
  'z': 'POS',
  'q': 'POS',
  'u': 'POS',
  'i': 'POS',
  'e': 'POS',
  'r': 'POS',
  'd': 'POS'}}

In [9]:
def clean_instance(instance):
    ret = {k:instance[k] for k in ["titulo", "id"]}

    ret["labels"] = {}
    
    for k, label in instance["targets"].items():
        names = instance["mapping"][k]

        if isinstance(names, str):
            names = [names]

        for name in names:
            ret["labels"][name] = label
    return ret


clean_dataset = [clean_instance(instance) for instance in dataset]
    
#clean_dataset = [clean_instance(instance) for instance in dataset]
print("Cantidad de titulares: ", len(clean_dataset))
print("Cantidad de targets: ", sum([len(l["labels"])for l in clean_dataset]))



Cantidad de titulares:  1930
Cantidad de targets:  2236


Saquemos espacios espurios de 

## Sanity check

In [15]:
for instance in clean_dataset:
    for k, v in instance["labels"].items():
        if k not in instance["titulo"]:
            print(instance)
            print(k)

{'titulo': 'Clase media, empleo y pymes, los ejes del plan de gobierno de Macri', 'id': 2158, 'labels': {'Gobierno': 'POS', 'Macri': 'POS'}}
Gobierno
{'titulo': 'Fabiola Yáñez: "Este Gobierno no tiene empatía con nadie" | Entrevista exclusiva con la pareja de Alberto Fernández', 'id': 8131, 'labels': {'gobierno': 'NEG', 'Alberto Fernández': 'POS'}}
gobierno
{'titulo': 'Alberto Fernández advirtió que el país “está virtualmente defaulteado, el Gobierno dijo que no puede pagar”', 'id': 10748, 'labels': {'gobierno': 'NEG', 'Alberto Fernández': 'POS'}}
gobierno
{'titulo': 'Polémica entre el Gobierno y Alberto Fernández por la "ley mentira", el proyecto para ampliar el falso testimonio', 'id': 11474, 'labels': {'gobierno': 'NEG', 'Alberto Fernández': 'NEG'}}
gobierno
{'titulo': 'Cristina juega al silencio pero influye en la campaña y los proyectos de gobierno', 'id': 12462, 'labels': {'Gobierno': 'NEU'}}
Gobierno
{'titulo': 'Por error, y en plena veda, TN dio por ganador a Alberto Férnandez'

Nada grave